In [ ]:
results_name = 'SPY-from-2020-09-01-to-2022-08-01'
results_name = 'AAPL-sandbox'

In [ ]:
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 100

import sys; sys.path.append('../..')

from src.results import read_results
from src.risk import simulate_account

orders = list(read_results.get_orders(results_name))
simulation = simulate_account.Simulation.from_orders(orders, simulate_account.IdealAccountState.empty(simulate_account.build_td_simulation()))
len(orders)

In [ ]:
print(simulation.get_ideal_initial_balance())
simulation.get_values().plot()

# Activity

In [ ]:
import pandas as pd
from zoneinfo import ZoneInfo
def transaction(o):
    d = o.to_dict()
    del d['datetime']
    return d
transactions = pd.DataFrame([transaction(o) for o in orders], index=[o.datetime.astimezone(None) for o in orders])
transactions.index = pd.to_datetime(transactions.index, utc=True).tz_convert("America/New_York")


In [ ]:
from matplotlib import pyplot as plt
transactions['symbol'].groupby(by=[transactions.index.year, transactions.index.month]).count().plot()
plt.show()
transactions['symbol'].groupby(by=[transactions.index.year]).count().plot()
plt.show()
transactions['symbol'].groupby(by=[transactions.index.month]).count().plot()
plt.show()

# Quantstats Portfolio

In [ ]:
returns = simulation.get_returns()
returns

In [ ]:
import quantstats
risk_free_rate = 0.02
returns = simulation.get_returns()
quantstats.reports.full(returns, rf=risk_free_rate)

In [ ]:
quantstats.plots.rolling_volatility(returns, period=22, period_label="22d")

In [ ]:
quantstats.plots.rolling_sharpe(returns, rf=risk_free_rate, period=22, period_label="22d")

In [ ]:
quantstats.stats.drawdown_details(returns)

# Outlier Analysis

In [ ]:
import plotly.express as px
upside_outliers = quantstats.stats.outliers(returns)
px.scatter(upside_outliers).show()
upside_outliers.sort_values(ascending=False)

In [ ]:
downside_outliers = -quantstats.stats.outliers(-returns)
px.scatter(downside_outliers).show()
downside_outliers.sort_values(ascending=True)

# What If: Remove Outliers

In [ ]:

quantstats.reports.full(returns[[d not in downside_outliers and d not in upside_outliers for d in returns.index]], rf=risk_free_rate)